## Top repostitories for github topics


# Pick a website and describe your objective
- Browse through different sites and pick on to scrape. Check the "Project Ideas" section for inspiration.
- Identify the information you'd like to scrape from the site. Decide the format of the output CSV file.
- Summarize your project idea and outline your strategy in a Juptyer notebook. Use the "New" button above.

## project outline
 -we're going to scrape https://github.com/topics
 - we'll get a list of topics. for each topic title, topic page URL and topic descripition
 -for each topic, we'll get the top 25 repositories in the topic fromthe topic page
 -for each repository, we'll grab the  repo name, username, stars and repo URL
 -for each topic we'll create a CSV file in the following format.
 

# Use request library to download the webpages

In [99]:
import requests
topics_url = "https://github.com/topics"



In [100]:
response = requests.get(topics_url)

In [101]:
response.status_code

200

In [102]:
page_contents = response.text

In [ ]:
page_contents[:1000]

In [104]:
with open('webpage.html', "w", encoding="utf-8") as f:
    f.write(page_contents)

# Use BeautifulSoup to extract information

In [105]:
from bs4 import BeautifulSoup


In [106]:
doc = BeautifulSoup(page_contents,'html.parser')

In [ ]:
doc

In [108]:
p_tags = doc.find_all('p')

In [ ]:
p_tags[:5]

In [110]:
selection_class = "f3 lh-condensed mb-0 mt-1 Link--primary"
topic_title_tags = doc.find_all('p',{'class': selection_class})

In [111]:
len(topic_title_tags)

30

In [ ]:
topic_title_tags[:5]

In [113]:

description_class = "f5 color-fg-muted mb-0 mt-1"
topic_description_tag = doc.find_all('p',{'class': description_class})

In [114]:
len(topic_description_tag)

30

In [ ]:
topic_description_tag[:5]

In [116]:
topic_title_tag0 = topic_title_tags[0]

In [ ]:
div_tag = topic_title_tag0.parent
div_tag


In [118]:

topic_link_tags = doc.find_all('a',{'class':"no-underline flex-1 d-flex flex-column"})
len(topic_link_tags)

30

In [119]:
topic0_url = "https://github.com" + topic_link_tags[0]['href']
print(topic0_url)

https://github.com/topics/3d


In [120]:
topic1_url = "https://github.com" + topic_link_tags[1]['href']
print(topic1_url)

https://github.com/topics/ajax


In [ ]:
topic_title_tags[0].text


In [ ]:
topic_titles = []
for tag in topic_title_tags:
  topic_titles.append(tag.text)

print(topic_titles)

In [ ]:
topic_description = []
for tag in topic_description_tag:
  topic_description.append(tag.text.strip())

print(topic_description)

In [ ]:

topic_urls = []
base_urls = "https://github.com"
for tag in topic_link_tags:
  topic_urls.append(base_urls + tag['href'])

print(topic_urls)

# Create CSV file with extracted information

In [125]:
import pandas as pd

In [126]:
topics_dict = { 'title':topic_titles,
               'description':topic_description,
               'url':topics_url
               }

In [127]:
topic_df = pd.DataFrame(topics_dict)

In [ ]:
topic_df

In [129]:
topic_df.to_csv('topics.csv', index=None )

# Getting information out of a topic page

In [130]:
topic_page_url = topic_urls[0]


In [131]:
topic_page_url

'https://github.com/topics/3d'

In [132]:
response = requests.get(topic_page_url)

In [133]:
response.status_code
len(response.text)

455632

In [134]:
topic_doc = BeautifulSoup(response.text,'html.parser')

In [ ]:
topic_doc

In [136]:
h3_selection = "f3 color-fg-muted text-normal lh-condensed"
repo_tags = topic_doc.find_all('h3',{'class':h3_selection})

In [ ]:
len(repo_tags)
repo_tags[:5]

In [138]:
a_tags  = repo_tags[0].find_all('a')
a_tags[0].text.strip()

'mrdoob'

In [139]:
a_tags[1].text.strip()

'three.js'

In [140]:
repo_url = base_urls + a_tags[1]['href']
print(repo_url)

https://github.com/mrdoob/three.js


In [141]:
star_selection = "Counter js-social-count"
star_tag = topic_doc.find_all('span',{"class":star_selection})


In [142]:
len(star_tag)
star_tag[0].text

'89.1k'

In [143]:
def parse_star_count(star_str):
  star_str = star_str.strip()
  if star_str[-1] ==  'k':
   return  int(float(star_str[:-1])*1000)
  return int(star_str)

In [144]:
parse_star_count(star_tag[0].text)

89100

In [145]:
def get_repo_info(h3_tag, star_tag):
  a_tags = h3_tag.find_all('a')
  username = a_tags[0].text.strip()
  repo_name = a_tags[1].text.strip()
  repo_url = base_urls + a_tags[1]['href']
  stars = parse_star_count(star_tag.text.strip())
  return username,repo_name,stars,repo_url

-- repo_tags are all h3 tags..
 --  star_tags are all span tags

In [146]:
get_repo_info(repo_tags[0], star_tag[0])

('mrdoob', 'three.js', 89100, 'https://github.com/mrdoob/three.js')

In [147]:
topic_repos_dict = {
    'username' : [],
    'repo_name' : [],
      'stars' : [],
    'repo_url' : []

}

for i in range(len(repo_tags)):
  repo_info = get_repo_info(repo_tags[i], star_tag[i])
  topic_repos_dict['username'].append(repo_info[0])
  topic_repos_dict['repo_name'].append(repo_info[1])
  topic_repos_dict['stars'].append(repo_info[2])
  topic_repos_dict['repo_url'].append(repo_info[3])


In [ ]:
topic_repos_dict

In [149]:
topic_repos_df = pd.DataFrame(topic_repos_dict)

In [ ]:
topic_repos_df

# FINAL CODE

In [151]:
# above function we created is only includes the '3d' topic repositories to
# make it for all the topics we have to create funciton

import os

def get_topic_page(topic_url):
    #download the page
  response = requests.get(topic_url)
  
  #check successfull response
  if response.status_code != 200:
     raise Exception('Failed to load page {}'.format(topic0_url))
  
  #parse using beautifulsoup 
  topic_doc = BeautifulSoup(response.text, 'html.parser')
  return topic_doc


def get_repo_info(h3_tag, star_tag):
         a_tags = h3_tag.find_all('a')
         username = a_tags[0].text.strip()
         repo_name = a_tags[1].text.strip()
         repo_url = base_urls + a_tags[1]['href']
         stars = parse_star_count(star_tag.text.strip())
         return username,repo_name,stars,repo_url

def get_topic_repos(topic_doc):

  #get h3 tag containing repo title, repo url and username
  h3_selection = "f3 color-fg-muted text-normal lh-condensed"
  repo_tags = topic_doc.find_all('h3',{'class':h3_selection})

  #get star tag
  star_selection = "Counter js-social-count"
  star_tag = topic_doc.find_all('span',{"class":star_selection})
   #get repo info

  topic_repos_dict = { 'username' : [], 'repo_name' : [], 'stars' : [], 'repo_url' : []    }

  for i in range(len(repo_tags)):
       repo_info = get_repo_info(repo_tags[i], star_tag[i])
       topic_repos_dict['username'].append(repo_info[0])
       topic_repos_dict['repo_name'].append(repo_info[1])
       topic_repos_dict['stars'].append(repo_info[2])
       topic_repos_dict['repo_url'].append(repo_info[3])
        
  return pd.DataFrame(topic_repos_dict)

def scrape_topic(topic_url, path):
  if os.path.exists(path):
    print("The file {} already exists. Skipping ..." .format(path))
    return
  topic_df = get_topic_repos(get_topic_page(topic_url))
  topic_df.to_csv(path, index=None)



In [152]:
topic_urls[4]

'https://github.com/topics/android'

In [ ]:
get_topic_repos(get_topic_page(topic_urls[4]))



# Write a single function to:

 - Get the list of topics from the topics page

 - Get the list of top repos from the individual topic pages

 - For each topic, crate a CSV of the top repos for the topic

In [154]:

def get_topic_titles(doc):
   selection_class = "f3 lh-condensed mb-0 mt-1 Link--primary"
   topic_title_tags = doc.find_all('p',{'class': selection_class}) 
  
   topic_titles = []
   for tag in topic_title_tags:
          topic_titles.append(tag.text) 
   return topic_titles

def get_topic_description(doc):
    description_class = "f5 color-fg-muted mb-0 mt-1"
    topic_description_tag = doc.find_all('p',{'class': description_class})

    topic_description = []
    for tag in topic_description_tag:
          topic_description.append(tag.text.strip())
    return topic_description
      
def get_topic_urls(doc): 
  topic_link_tags = doc.find_all('a',{'class':"no-underline flex-1 d-flex flex-column"})
  topic_urls = []
  base_urls = "https://github.com"

  for tag in topic_link_tags:
        topic_urls.append(base_urls + tag['href'])

  return topic_urls

def scrape_topics():
  topics_url = "https://github.com/topics"
  response = requests.get(topics_url)
  if response.status_code != 200:
     raise Exception('Failed to load page {}'.format(topics_url))

  topics_dict = {
      'title' : get_topic_titles(doc),
      'description' : get_topic_description(doc),
      'url': get_topic_urls(doc)
  }
  return pd.DataFrame(topics_dict)


In [ ]:
scrape_topics()

In [156]:
def scrape_topic_repos():
  print('Scraping list of topics')
  topics_df = scrape_topics()

  os.makedirs('data',exist_ok = True)
  for index, row in topics_df.iterrows():
    print('Scraping top repositories for "{}"'.format(row['title']))
    scrape_topic(row['url'], 'data/{}.csv'.format(row['title']))

In [ ]:
scrape_topic_repos()